In [1]:
import json
import spacy
import random
from spacy.util import minibatch, compounding
from pathlib import Path
from spacy.training.example import Example

f = open('annual_report_23.json', encoding="utf-8")
data = json.load(f)

countEn = 0
countText = 0
training_set = []
for p in data:
    if p["language"] == "zh":
        countEn = countEn + 1
        text_list = []
        for pp in p["text"]:
            #print(pp["text"])
            countText = countText + 1
            if len(pp["text"]):
                text_list.append(pp["text"])
            #print(pp["ner"])
            ner_str = json.loads(pp["ner"])
            entities_list = []
            for ner in ner_str:
                #ner = json.loads(ner_str)
                #print(ner)
                if len(ner):
                    #print(ner["text"])
                    entities_list.append(tuple((ner["start"],ner["end"],ner["label"])))
            print(entities_list)
            if len(entities_list):
                training_set.append(tuple((pp["text"],{"entities":entities_list})))
print(countEn)
print(countText)
print(len(training_set))

f.close()

nlp=spacy.load("zh_core_web_sm") 
ner=nlp.get_pipe('ner')

for _, annotations in training_set:
  for ent in annotations.get("entities"):
    ner.add_label(ent[2])

# Resume training
optimizer = nlp.resume_training()
move_names = list(ner.move_names)

# List of pipes you want to train
pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]

# List of pipes which should remain unaffected in training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
with nlp.disable_pipes(*other_pipes) :

  # Training for 30 iterations     
  for itn in range(30):
    # shuffle examples before training
    random.shuffle(training_set)
    # batch up the examples using spaCy's minibatch
    batches = minibatch(training_set, size=compounding(4.0, 32.0, 1.001))
    # ictionary to store losses
    losses = {}
    for batch in batches:
        for text, annotations in batch:
            # create Example
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotations)
            # Update the model
            nlp.update([example], losses=losses, drop=0.5)
            print("Losses", losses)

# Save the  model to directory
output_dir = Path('output_zh/')
nlp.to_disk(output_dir)
print("Saved model to", output_dir)


[]
[(0, 3, 'PERSON'), (4, 7, 'DATE'), (13, 20, 'DATE'), (28, 36, 'DATE')]
[(6, 9, 'LOC'), (10, 12, 'GPE'), (19, 23, 'DATE')]
[(8, 18, 'ORG'), (20, 22, 'ORG'), (37, 39, 'ORG'), (43, 51, 'ORG'), (70, 72, 'DATE'), (75, 80, 'ORG'), (82, 84, 'DATE')]
[(3, 7, 'ORG'), (13, 17, 'ORG'), (25, 33, 'LOC')]
[(7, 19, 'LOC'), (21, 41, 'LOC'), (45, 72, 'LOC'), (74, 88, 'LOC')]
[(0, 3, 'PERSON'), (4, 8, 'DATE'), (14, 21, 'DATE'), (29, 36, 'DATE')]
[(7, 9, 'NORP'), (10, 12, 'NORP'), (23, 27, 'DATE'), (32, 34, 'ORG'), (35, 40, 'CARDINAL')]
[(5, 14, 'DATE'), (27, 34, 'DATE'), (48, 53, 'DATE'), (55, 57, 'ORG'), (73, 81, 'DATE'), (98, 105, 'DATE'), (116, 121, 'DATE'), (123, 132, 'ORG'), (132, 152, 'LOC'), (154, 159, 'DATE'), (163, 169, 'ORG'), (171, 173, 'ORG')]
[]
[(0, 3, 'PERSON'), (4, 8, 'DATE'), (14, 21, 'DATE')]
[(13, 16, 'DATE'), (59, 66, 'DATE'), (84, 91, 'DATE')]
[(5, 15, 'DATE'), (17, 30, 'ORG'), (40, 44, 'ORG'), (44, 47, 'DATE'), (55, 57, 'LOC'), (58, 60, 'LOC'), (61, 63, 'LOC'), (76, 86, 'ORG')]


In [2]:

# Load the saved model and predict
print("Loading from", output_dir)
nlp_updated = spacy.load(output_dir)
doc = nlp_updated("Fridge can be ordered in FlipKart" )
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

# Testing the model
doc = nlp("143新鴻基地產發展有限公司二○一九至二○年年報")
print("Entities", [(ent.text, ent.label_) for ent in doc.ents])

Loading from output_zh
Entities [('Fridge can', 'LOC')]
Entities [('新鴻基地產發展有限公司', 'ORG'), ('二○一九至二○年', 'DATE')]
